# Network Topology Analysis

Demonstrates graph-theoretic analysis of power system networks using the
`Network` application module.

Topics covered:
- Bus-to-index mapping
- Incidence matrix construction
- Weighted Laplacians (length, impedance, delay)
- Branch parameter distributions
- Spectral analysis of network structure

In [ ]:
import numpy as np
from scipy.sparse.linalg import eigsh
from esapp import GridWorkBench
from esapp.components import Branch, Bus
from esapp.apps.network import BranchType
from esapp.utils import format_plot, sorteig

In [2]:
# This cell is hidden in the documentation.
import ast

with open('../data/case.txt', 'r') as f:
    case_path = ast.literal_eval(f.read().strip())

wb = GridWorkBench(case_path)

'open' took: 4.0442 sec


In [ ]:
# Plotting functions (hidden from documentation)
import sys; sys.path.insert(0, "..")
from plot_helpers import (
    plot_incidence_and_degree, plot_spy_matrices,
    plot_histograms, plot_eigenspectrum, plot_fiedler,
)

## 1. Bus Mapping and Incidence Matrix

The `busmap()` provides the mapping from PowerWorld bus numbers to matrix indices.
The incidence matrix has one row per branch with +1/-1 entries.

In [3]:
bmap = wb.network.busmap()
print(f"Bus count: {len(bmap)}")
print(f"First 5 mappings:")
print(bmap.head())

A = wb.network.incidence()
print(f"\nIncidence matrix: {A.shape} (branches x buses)")
print(f"Non-zeros: {A.nnz}")

Bus count: 37
First 5 mappings:
BusNum
1    0
2    1
3    2
4    3
5    4
dtype: int64

Incidence matrix: (89, 37) (branches x buses)
Non-zeros: 178


In [ ]:
plot_incidence_and_degree(A)

## 2. Weighted Laplacians

The graph Laplacian L = A.T @ W @ A captures network connectivity with different
weighting schemes: inverse squared length, inverse impedance, or inverse squared delay.

In [ ]:
L_len = wb.network.laplacian(BranchType.LENGTH)
L_res = wb.network.laplacian(BranchType.RES_DIST)

print(f'Length-weighted Laplacian: {L_len.shape}, nnz={L_len.nnz}')
print(f'Impedance-weighted Laplacian: {L_res.shape}, nnz={L_res.nnz}')

plot_spy_matrices([L_len, L_res],
                  ['Length-Weighted Laplacian', 'Impedance-Weighted Laplacian'],
                  colors=['steelblue', 'tomato'])

## 3. Branch Parameters

Examine the distributions of branch lengths, impedance magnitudes, and
propagation delays.

In [ ]:
lengths = wb.network.lengths()
zmag = wb.network.zmag()

plot_histograms([lengths, zmag],
                ['Branch Length Distribution', 'Impedance Magnitude Distribution'],
                ['Length (km)', '|Z| (pu)'],
                colors=['steelblue', 'tomato'])

print(f'Length range: [{lengths.min():.3f}, {lengths.max():.3f}] km')
print(f'|Z| range:    [{zmag.min():.6f}, {zmag.max():.6f}] pu')

## 4. Spectral Analysis

The eigenvalues of the Laplacian encode the network's structural properties.
The algebraic connectivity (second-smallest eigenvalue) measures how well-connected
the network is.

In [ ]:
k = min(10, L_len.shape[0] - 1)
vals_len, vecs_len = eigsh(L_len.astype(float), k=k, which='SM')
vals_len, vecs_len = sorteig(vals_len, vecs_len)

vals_res, vecs_res = eigsh(L_res.astype(float), k=k, which='SM')
vals_res, vecs_res = sorteig(vals_res, vecs_res)

plot_eigenspectrum([vals_len, vals_res],
                   ['Length-Weighted Eigenvalues', 'Impedance-Weighted Eigenvalues'])

print(f'Algebraic connectivity (length): {vals_len[1]:.6f}')
print(f'Algebraic connectivity (impedance): {vals_res[1]:.6f}')

## 5. Fiedler Vector Visualization

The Fiedler vector (eigenvector of the second-smallest eigenvalue) reveals the
natural partition of the network into two clusters.

In [ ]:
fiedler = vecs_len[:, 1]
plot_fiedler(fiedler)

## Summary

Network topology analysis provides:
- **Incidence matrix**: Branch-bus connectivity (signed)
- **Laplacians**: Weighted connectivity matrices for spectral analysis
- **Branch parameters**: Physical characteristics of each branch
- **Spectral decomposition**: Reveals structural properties and natural clusters